In [ ]:
# %pip install tf-agents==0.14.0

In [1]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.environments.gym_wrapper import GymWrapper

from utils import make_env


env = make_env(max_step=1000)
env = GymWrapper(env)
env = TFPyEnvironment(env)

env.reset()
_ = env.render()

In [2]:
train_iters = 5_000_000
buffer_size = 1_000_000

update_period = 10 # run a training step every 10 collect steps
episode_max_step = 1_000

In [3]:
import tensorflow as tf
from tensorflow.keras import layers

from tf_agents.networks.q_network import QNetwork


preprocessing_layer = layers.Lambda(lambda obs: tf.cast(obs, tf.float32) / 4.)
fc_layer_params = [128, 64, 64]
dropout_layer_params = [0.5, 0.5, 0.5]

q_net = QNetwork(
    env.observation_spec(),
    env.action_spec(),
    preprocessing_layers=preprocessing_layer,
    fc_layer_params=fc_layer_params,
    dropout_layer_params=dropout_layer_params
)

In [4]:
from tensorflow.keras import optimizers, losses

from tf_agents.agents.dqn.dqn_agent import DqnAgent


train_step = tf.Variable(0)

optimizer = optimizers.Adam(learning_rate=3e-7)
                                     
epsilon_fn = optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0,
    decay_steps=train_iters // update_period,
    end_learning_rate=0.01
)

agent = DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    target_update_period=2_000,
    td_errors_loss_fn=losses.Huber(reduction="none"),
    gamma=0.95, # discount factor
    train_step_counter=train_step,
    epsilon_greedy=lambda: epsilon_fn(train_step)
)

agent.initialize()

In [5]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer


replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=buffer_size
)
replay_buffer_observer = replay_buffer.add_batch

dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2,
    num_parallel_calls=3
).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [6]:
from tf_agents.metrics import tf_metrics
from tf_agents.eval.metric_utils import log_metrics

import logging


class ShowProgress:

    def __init__(self, total):
        self.counter = 0
        self.total = total
        
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")
            

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric(),
]

logging.getLogger().setLevel(logging.INFO)
log_metrics(train_metrics)

INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 0
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


In [7]:
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver


# class LimitSteps:

#     def __init__(self, max):
#         self.max = max
#         self.counter = 0
        
#     @tf.function
#     def __call__(self, trajectory):
#         # if not trajectory.is_boundary():
#         #     self.counter += 1
#         # else:
#         #     self.counter = 0

#         if self.counter > self.max:
#             env.reset()


collect_driver = DynamicStepDriver(
    env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + train_metrics,
    num_steps=update_period
)

In [8]:
from tf_agents.policies.random_tf_policy import RandomTFPolicy


initial_collect_policy = RandomTFPolicy(
    env.time_step_spec(),
    env.action_spec()
)

init_driver = DynamicStepDriver(
    env,
    initial_collect_policy,
    observers=[replay_buffer.add_batch, ShowProgress(buffer_size//4)],
    num_steps=buffer_size//4
)

final_time_step, final_policy_state = init_driver.run()

250000/250000

In [9]:
from tf_agents.utils.common import function


collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [10]:
def train_agent(n_iterations):
    best_average_return = 0

    time_step = None
    policy_state = agent.collect_policy.get_initial_state(env.batch_size)

    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        
        train_loss = agent.train(trajectories)

        print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")

        if iteration % (n_iterations//1000) == 0:
            log_metrics(train_metrics)

        if train_metrics[2].result() > best_average_return:
            best_average_return = train_metrics[2].result()
            print(f"Saving best model with: Average Return of {best_average_return} in iters of {iteration}")
            tf.saved_model.save(agent._q_network, f"models/tf-agents/logs/Snake DQN TF-Agents ({n_iterations} iters) with rwd#{best_average_return}")

    log_metrics(train_metrics)
    tf.saved_model.save(agent._q_network, f"models/tf-agents/Snake DQN TF-Agents ({n_iterations} iters)")

In [11]:
train_agent(n_iterations=train_iters)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 10
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


4999 loss:0.45013

INFO:absl: 
		 NumberOfEpisodes = 138
		 EnvironmentSteps = 50010
		 AverageReturn = -315.5895080566406
		 AverageEpisodeLength = 395.79998779296875


9998 loss:1.05939

...

INFO:absl: 
		 NumberOfEpisodes = 8110
		 EnvironmentSteps = 3850010
		 AverageReturn = -222.48873901367188
		 AverageEpisodeLength = 912.0999755859375


389998 loss:0.15912

INFO:absl: 
		 NumberOfEpisodes = 8167
		 EnvironmentSteps = 3900010
		 AverageReturn = -194.76913452148438
		 AverageEpisodeLength = 767.4000244140625


390671 loss:0.15823

KeyboardInterrupt: 

In [13]:
frames = []

env.pyenv.envs[0].reset()

def save_frames(trajectory):
    global frames
    frames.append(env.pyenv.envs[0].render())

watch_driver = DynamicStepDriver(
    env,
    agent.policy,
    observers=[save_frames, ShowProgress(2_000)],
    num_steps=2_000)
final_time_step, final_policy_state = watch_driver.run()

2000/2000

In [14]:
import PIL

import os


image_path = os.path.join("rl videos", f"snake_tfagents_{train_iters}.gif")

frame_images = []
for frame in frames:
    frame_images.append(PIL.Image.fromarray(frame))

frame_images[0].save(image_path, format='GIF',
                     append_images=frame_images[1:],
                     save_all=True,
                     duration=30,
                     loop=0)